Hier findet die Analyse der Daten statt. zunächst laden wir die Daten herein.
und danach gucken wir uns verschiedene korrelationen an.
(irgendwas mit PCA)

In [1]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
country_list = ['China', 'United Kingdom', 'France', 'Russian Federation', 'Brazil', 'Japan', 'Germany', 'United States', 'India',
                 'Canada', 'Argentina', 'Australia', 'Indonesia', 'Mexico', 'Turkiye', 'Korea, Rep.']

indize_list = {'Shenzhen Index' : '399001.SZ', 'FTSE 100 Index' : '^FTSE', 'MOEX Russia Index' : 'IMOEX.ME', 'Bovespa Index' : '^BVSP',
                 'Nikkei 225' : '^N225', 'S&P500' : '^GSPC', 'TSX Composite Index' : '^GSPTSE', 'NSE Nifty 50' : '^NSEI', 
                 'MERVAL Index' : '^MERV', 'ASX 200' : '^AXJO', 'IDX Composite Index' : '^JKSE', 'BMV IPC' : '^MXX', 
                 'KOSPI Index' : '^KS11'}

#name_list = [('China', 'Shenzhen Index'), ('Germany', 'DAX'), ('France', 'CAC 40 Index'), ('United Kingdom', 'FTSE 100 Index'), 
 #               ('Russian Federation', 'MOEX Russia Index'), ('Brazil', 'Bovespa Index'), ('Japan', 'Nikkei 225'), 
  #              ('United States', 'S&P500'), ('India', 'NSE Nifty 50')]

name_list = [('Brazil', 'Bovespa Index'), ('United Kingdom', 'FTSE 100 Index'), ('India', 'NSE Nifty 50'),('Germany', 'DAX')]

path_indize = r'C:\Users\arne\Desktop\Informatik\Data Science Projekt\Data\indize_data'
path_indicator = r'C:\Users\arne\Desktop\Informatik\Data Science Projekt\Data\indicator_data\extracted_data' 

feature_list = ['GDP_growth', 'CO2_emissions', 'inflation_rate', 'unemployement_rate', 'individuals_using_internet']

#test_feature = 'GDP_growth'
#test_feature = 'CO2_emissions'
#test_feature = 'GDP'
#test_feature = 'inflation_rate'
#test_feature = 'unemployement_rate'
#test_feature = 'individuals_using_internet'

In [3]:
# read data
def readData(file_path):
    return pd.read_excel(file_path)

In [4]:
#erwartet eine Liste von Dates
#zurück kommt ein dict mit jahr als key und einem tupel mit max und dann min date {year : (max,min)}
def get_min_and_max_date(data):
    # Konvertiere die Datumssrings in Datetime-Objekte und gruppiere sie nach Jahr
    daten_liste_datum = [datetime.strptime(datum, '%Y-%m-%d') for datum in data]
    gruppierte_daten = {}
    for datum in daten_liste_datum:
        jahr = datum.year 
        if jahr not in gruppierte_daten:
            gruppierte_daten[jahr] = [datum]
        else:
            gruppierte_daten[jahr].append(datum)

    # Finde das erste und letzte Datum für jedes Jahr
    erg = {}
    for jahr in gruppierte_daten.keys():
        erg[jahr] = (max(gruppierte_daten[jahr]), min(gruppierte_daten[jahr])) 
        
    return erg


In [5]:
# Daten kommen als pandas dataframe wir berechnen pro jahr das wachstum vom ersten zum letzten Tag
# gibt ein pandas dataframe zurück, in dem pro jahr das wachstum zu sehen ist
def calc_indize_growth_per_year(data):
    values = get_min_and_max_date(data['date'])

    df = pd.DataFrame(columns=['year', 'value'])
    index = 0

    for year in values.keys():
        (max, min) = values[year]
        index_max = (data['date'] == max.strftime('%Y-%m-%d')).idxmax() 
        index_min = (data['date'] == min.strftime('%Y-%m-%d')).idxmax()

        value_max = data['close'][index_max]
        value_min = data['close'][index_min]

        #this is for growth in % 
        #df.loc[index] = [year, (value_max/value_min - 1) * 100]
        
        df.loc[index] = [year, (value_max+value_min) / 2]
        index += 1
    
    return df
    

In [6]:
# erwartet ein dict von mehreren panda dataframes und normt die Datensätze
def cut_years(data):
    maximales_min_year = 100
    minimales_max_year = 3000
    
    for feature in data.keys():
        min_year = min(data[feature]['year'])
        max_year = max(data[feature]['year'])

        if min_year > maximales_min_year:
            maximales_min_year = min_year
        if max_year < minimales_max_year:
            minimales_max_year = max_year
    
    for feature in data.keys():
        data[feature] = data[feature].query(f'{maximales_min_year} <= year <= {minimales_max_year}')
    
    return data



In [7]:
def make_plot(data_dict):
    categories = feature_list

    fig = go.Figure()

    for country in data_dict.keys():
        # Suchen Sie das Tupel, in dem der erste Eintrag gleich dem Land ist, um den Namen des zugehörigen Index zu erhalten
        entry = next((tupel for tupel in name_list if tupel[0] == country), None)
        aktienindex = data_dict[country][entry[1]]

        country_correlation_coefficients = []

        for feature in feature_list:
            feature_data = data_dict[country][feature]

            # Berechnen Sie den Korrelationskoeffizient
            correlation_coefficient = np.corrcoef(feature_data['value'], aktienindex['value'])[0, 1] * 5
            country_correlation_coefficients.append(correlation_coefficient)

        fig.add_trace(go.Scatterpolar(
            r=country_correlation_coefficients,
            theta=categories,
            fill='toself',
            name=country,
            showlegend=True
        ))


    fig.update_layout(
    polar=dict(
        radialaxis=dict(
        visible=True,
        range=[-5, 5]
        )),
    showlegend=True
    )

In [8]:

# hat die form: {'country' : {indizename : {}, GDP_growth : {}, }}
#hier werden die Daten geladen
data_dict = {}
#dict_indi
for (country,index) in name_list:
    country_dict = {}

    for feature in feature_list:
        file_path = path_indicator + '\\' + country + '_' + feature + '.xlsx'    
        country_dict[feature] = readData(file_path)[::-1].reset_index(drop=True)

    file_path = path_indize + '\\' + index + '.xlsx'
    country_dict[index] = calc_indize_growth_per_year(readData(file_path))



    data_dict[country] = cut_years(country_dict)

make_plot(data_dict)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\arne\\Desktop\\Informatik\\Data Science Projekt\\Data\\indicator_data\\extracted_data\\Brazil_GDP_growth.xlsx'

In [39]:
#erstmal nur ein radardiagramm anzeigen lassen
